# Análise API

## Integração com APIs: Feriados e Tempo



### Utilize as APIs públicas abaixo para responder às questões 1-8:
- [Public Holiday API](https://date.nager.at/Api)
- [Open-Meteo Historical Weather API](https://open-meteo.com/)

1. **Quantos feriados há no Brasil em todo o ano de 2024?**

2. **Qual mês de 2024 tem o maior número de feriados?**

3. **Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

4. **Qual foi a temperatura média em cada mês?**  
   Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024.  
   
5. **Qual foi o tempo predominante em cada mês nesse período?**  
   Utilize como referência para o código de tempo (_weather_code_) o seguinte link: [WMO Code](https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c).

6. **Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

7. **Considere as seguintes suposições:**
   - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
   - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
   - O cidadão carioca só vai à praia quando não está com frio;
   - O cidadão carioca também só vai à praia em dias com sol, evitando dias **totalmente** nublados ou chuvosos (considere _weather_code_ para determinar as condições climáticas).

   Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

8. **Qual foi o feriado "mais aproveitável" de 2024?**  
   Considere o melhor par tempo e temperatura.

## Imports

In [2]:
import json
import requests
import pandas as pd
from utils.utils import Weather # Importa a dlasse Weather criada em utils.py que consome a API Open-Meteo Historical Weather API

# Gera instância de Weather 
wrio = Weather(-22.914469232838528, -43.44618954745923) # Coordenada CENTRO calculada utilizando geopandas

### Dados de feriados

In [3]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)
holidays = pd.DataFrame(public_holidays)
holidays

,date,localName,name,countryCode,fixed,global,counties,launchYear,types
0,2024-01-01,Confraternização Universal,New Year's Day,BR,False,True,None,None,[Public]
1,2024-02-12,Carnaval,Carnival,BR,False,True,None,None,"[Bank, Optional]"
2,2024-02-13,Carnaval,Carnival,BR,False,True,None,None,"[Bank, Optional]"
3,2024-03-29,Sexta-feira Santa,Good Friday,BR,False,True,None,None,[Public]
4,2024-03-31,Domingo de Páscoa,Easter Sunday,BR,False,True,None,None,[Public]
5,2024-04-21,Dia de Tiradentes,Tiradentes,BR,False,True,None,None,[Public]
6,2024-05-01,Dia do Trabalhador,Labour Day,BR,False,True,None,None,[Public]
7,2024-05-30,Corpus Christi,Corpus Christi,BR,False,True,None,None,[Public]
8,2024-07-09,Revolução Constitucionalista de 1932,Constitutionalist Revolution of 1932,BR,False,False,[BR-SP],None,[Public]
9,2024-09-07,Dia da Independência,Independence Day,BR,False,True,None,None,[Public]


### Tratativas

In [4]:
# dicionário para converter os meses de integer para string em Português
meses = {
    1 : 'Janeiro',
    2 : 'Fevereiro',
    3 : 'Março',
    4 : 'Abril',
    5 : 'Maio',
    6 : 'Junho',
    7 : 'Julho',
    8 : 'Agosto',
    9 : 'Setembro',
    10: 'Outubro',
    11: 'Novembro',
    12: 'Dezembro'
}
# dicionário para converter os dias de integer para string em Português
dias_semana = {
    0: "Segunda-feira",
    1: "Terça-feira",
    2: "Quarta-feira",
    3: "Quinta-feira",
    4: "Sexta-feira",
    5: "Sábado",
    6: "Domingo"
}
# Novas colunas
holidays['date'] = pd.to_datetime(holidays['date']) # Converte a coluna date para datetime
holidays['month'] = holidays['date'].apply(lambda x: x.month).map(meses) # Extrai o mês da coluna date e transcreve para o Português
holidays['weekday_n'] = holidays['date'].apply(lambda x: x.weekday()) # Extrai o dia da semana da coluna date como um inteiro 
holidays['weekday'] = holidays['date'].apply(lambda x: x.weekday()).map(dias_semana) # Transcreve o dia da semana em Português



holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         14 non-null     datetime64[ns]
 1   localName    14 non-null     object        
 2   name         14 non-null     object        
 3   countryCode  14 non-null     object        
 4   fixed        14 non-null     bool          
 5   global       14 non-null     bool          
 6   counties     1 non-null      object        
 7   launchYear   0 non-null      object        
 8   types        14 non-null     object        
 9   month        14 non-null     object        
 10  weekday_n    14 non-null     int64         
 11  weekday      14 non-null     object        
dtypes: bool(2), datetime64[ns](1), int64(1), object(8)
memory usage: 1.2+ KB


1. **Quantos feriados há no Brasil em todo o ano de 2024?**

In [5]:
print(f'Resposta: Há {len(holidays)} feriados no Brasil. Obs.: Carnaval está sendo contado duas vezes pois são dois dias de feriado.')

Resposta: Há 14 feriados no Brasil. Obs.: Carnaval está sendo contado duas vezes pois são dois dias de feriado.


2. **Qual mês de 2024 tem o maior número de feriados?**

In [19]:
feriados_by_month = holidays.groupby('month').agg({'date': 'count'}).reset_index()
months_list = holidays['month'].unique().tolist()


feriados_by_month['month'] = pd.Categorical(feriados_by_month['month'], categories=months_list, ordered=True)

# Ordenando o DataFrame agrupado
feriados_ordenados_mes = feriados_by_month.sort_values('month').reset_index(drop=True)

feriados_ordenados_mes


,month,date
0,Janeiro,1
1,Fevereiro,2
2,Março,2
3,Abril,1
4,Maio,2
5,Julho,1
6,Setembro,1
7,Outubro,1
8,Novembro,2
9,Dezembro,1


In [6]:
holidays['month'].value_counts()

month
Fevereiro    2
Março        2
Novembro     2
Maio         2
Janeiro      1
Abril        1
Julho        1
Setembro     1
Outubro      1
Dezembro     1
Name: count, dtype: int64

3. **Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

In [7]:
feriados_na_semana = sum(holidays['weekday_n'].apply(lambda x: True if x in range(0, 5) else False))
print(f'Há {feriados_na_semana} feriados em dias de semana(segunda a sexta-feira) em 2024')

Há 9 feriados em dias de semana(segunda a sexta-feira) em 2024


4. **Qual foi a temperatura média em cada mês?**  
   Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024. 

In [30]:
# Utiliza o método forecast de Weather que recebe uma data de inicio e fim de um período de dias para obter os dados de temperatura de cada dia
temperaturas_diarias_2024 = wrio.forecast('2024-01-01', '2024-08-01') # retorna um dataframe com data, clima(tempo) e temperatura média

# Cria novas colunas 
temperaturas_diarias_2024['month_n'] = temperaturas_diarias_2024['data'].apply(lambda x: x.month) # Extrai o mês de cada registro como um valor inteiro
temperaturas_diarias_2024['month'] = temperaturas_diarias_2024['data'].apply(lambda x: x.month).map(meses) # Transcreve o mês de inteiro para os nomes dos meses em Português

# Agrupa os dados por mês e agrega a média de temperatura média de cada mês, ordena os meses de Janeiro a Dezembro
by_month = temperaturas_diarias_2024.groupby(['month_n','month']).agg({'temperatura media':'mean'}).sort_values('month_n', ascending=True).reset_index()

by_month[['month', 'temperatura media']]

,month_n,month,temperatura media
0,1,Janeiro,23.603844
1,2,Fevereiro,23.957384
2,3,Março,23.030052
3,4,Abril,22.043333
4,5,Maio,21.597729
5,6,Junho,19.782360
6,7,Julho,17.998266
7,8,Agosto,17.321251


5. **Qual foi o tempo predominante em cada mês nesse período?**  
   Utilize como referência para o código de tempo (_weather_code_) o seguinte link: [WMO Code](https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c).

In [29]:
# Arupa os dados do dataframe por mês e aplica o método que identifica a moda de uma Series
clima_predominante = temperaturas_diarias_2024.groupby(['month_n','month'])['clima'].apply(lambda x: x.mode()[0]).reset_index()
clima_predominante = clima_predominante[['month', 'clima']]
clima_predominante

,month,clima
0,Janeiro,Rain
1,Fevereiro,Light Drizzle
2,Março,Light Drizzle
3,Abril,Light Drizzle
4,Maio,Cloudy
5,Junho,Sunny
6,Julho,Cloudy
7,Agosto,Cloudy


6. **Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

In [9]:
# Novamente utilizando o dataframe "holidays" criado no início deste notebook vamos filtrar a data para que não tenha registros após a data de 01/08/2024
# Aplica o método forecast em cima de cada registro para obter as informações de temperatura e clima(tempo)
holidays['temperatura media'] = holidays[holidays['date'] <= '2024-08-01']['date'].apply(lambda x: wrio.forecast(x.date(), x.date())['temperatura media']) # Temperatura média do dia
holidays['clima'] = holidays[holidays['date'] <= '2024-08-01']['date'].apply(lambda x: wrio.forecast(x.date(), x.date())['clima']) # Clima do dia
holidays[holidays['date'] <= '2024-08-01'][['date', 'localName', 'temperatura media', 'clima']] # Filtra o dataframe para exibir apenas o período correto e colunas relevantes para a resposta

,date,localName,temperatura media,clima
0,2024-01-01,Confraternização Universal,24.900084,Light Drizzle
1,2024-02-12,Carnaval,30.202169,Mainly Sunny
2,2024-02-13,Carnaval,30.577166,Light Drizzle
3,2024-03-29,Sexta-feira Santa,25.089666,Light Rain
4,2024-03-31,Domingo de Páscoa,24.577164,Drizzle
5,2024-04-21,Dia de Tiradentes,23.133413,Partly Cloudy
6,2024-05-01,Dia do Trabalhador,28.154251,Sunny
7,2024-05-30,Corpus Christi,20.825081,Light Drizzle
8,2024-07-09,Revolução Constitucionalista de 1932,21.533417,Drizzle


7. **Considere as seguintes suposições:**
   - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
   - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
   - O cidadão carioca só vai à praia quando não está com frio;
   - O cidadão carioca também só vai à praia em dias com sol, evitando dias **totalmente** nublados ou chuvosos (considere _weather_code_ para determinar as condições climáticas).

   Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

In [10]:
# Função que retorna True para dias parcialmente nublados ou "melhores", e false para todos os outros(dias com garoa, chuva ou nublados são considerados false)
def clima_aproveitavel(clima):
    clima_ok = ['Mainly Sunny', 'Sunny', 'Partly Cloudy']
    if clima in clima_ok:
        return True
    else:
        return False

# Cria um novo dataframe que apenas os registros até 01/08/2024
holidays_2 = holidays[holidays['date'] <= '2024-08-01']

# Cria uma nova coluna de booleanos, True caso a temperatura seja maior ou igual a 20ºC e clima considerado aproveitável
holidays_2['praia'] = (holidays_2['temperatura media'] >= 20) & (holidays_2['clima'].apply(lambda x: clima_aproveitavel(x)))
holidays_2[holidays_2['praia'] == False][['date', 'localName', 'temperatura media', 'clima']] # Filtra somente os feriados não aproveitáveis

C:\Users\Johnny\AppData\Local\Temp\ipykernel_5360\2040352257.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays_2['praia'] = (holidays_2['temperatura media'] >= 20) & (holidays_2['clima'].apply(lambda x: clima_aproveitavel(x)))


,date,localName,temperatura media,clima
0,2024-01-01,Confraternização Universal,24.900084,Light Drizzle
2,2024-02-13,Carnaval,30.577166,Light Drizzle
3,2024-03-29,Sexta-feira Santa,25.089666,Light Rain
4,2024-03-31,Domingo de Páscoa,24.577164,Drizzle
7,2024-05-30,Corpus Christi,20.825081,Light Drizzle
8,2024-07-09,Revolução Constitucionalista de 1932,21.533417,Drizzle


8. **Qual foi o feriado "mais aproveitável" de 2024?**  
   Considere o melhor par tempo e temperatura.

In [11]:
holidays_2[holidays_2['praia'] == True][['date', 'localName', 'temperatura media', 'clima']]

,date,localName,temperatura media,clima
1,2024-02-12,Carnaval,30.202169,Mainly Sunny
5,2024-04-21,Dia de Tiradentes,23.133413,Partly Cloudy
6,2024-05-01,Dia do Trabalhador,28.154251,Sunny


O feriado mais aproveitável foi o dia do trabalhador, pois analisando sua temperatura, não estava muito mais baixa do que a do carnaval, porém o carnaval do dia 12/02/2024 não foi um dia "Completamente Ensolarado" e tem chances de ter ocorrido uma leve garoa, que foi o tempo predominante do dia seguinte.
<br>Levando isso em consideração, o Dia do Trabalhador além de ter sido um dia quente, também foi um dia belo e ensolarado.